In [28]:
from typing import Dict, List, Set, Union

import pandas as pd
from pandas import DataFrame

In [29]:
dataframe: DataFrame = pd.read_excel(
    "../../xlsx/logs_LTAT.03.001_20210207-0935_AT_01.xlsx",
    engine="openpyxl",
    dtype=object,
    header=0
)
dataframe_score: DataFrame = pd.read_excel(
    "../../xlsx/02-05 Hinded.xlsx",
    engine="openpyxl",
    dtype=object,
    header=0
)

In [30]:
def get_all_names(df: DataFrame) -> Set[str]:
    return set(df["Kasutaja täisnimi"])


def get_all_scored_students(df: DataFrame) -> Set[str]:
    all_scored_students = set()

    for _, row in df.iterrows():
        full_name = f"{row['Eesnimi']} {row['Perenimi']}"
        all_scored_students.add(full_name)

    return all_scored_students


def get_students(df: DataFrame, df_score: DataFrame) -> Set[str]:
    all_names = get_all_names(df)
    all_scored_students = get_all_scored_students(df_score)
    return all_names & all_scored_students


def get_components(df: DataFrame) -> List[str]:
    all_components = list(set(df["Komponent"]))
    all_components.sort()
    return all_components

In [31]:
import time
import datetime

def get_time(time_string: str) -> int:
    return int(
        time.mktime(
            datetime.datetime.strptime(
                time_string,
                "%d.%m.%Y %H:%M:%S"
            ).timetuple()
        )
    )


def get_end_time(df: DataFrame, students: Set[str], weeks: int) -> int:
    time_min = get_time_min(df, students)
    return time_min + 86400 * 7 * weeks


def get_time_min(df: DataFrame, students: Set[str]) -> int:
    time_list = [
        get_time(row["Aeg"]) for _, row in df.iterrows()
        if row["Kasutaja täisnimi"] in students
    ]
    return min(time_list)

In [32]:
def get_component_count(df: DataFrame, end_time: int, students: Set[str]) -> Dict[str, int]:
    count = {}

    for _, row in df.iterrows():
        the_time = get_time(row["Aeg"])

        if the_time > end_time:
            continue

        name = row["Kasutaja täisnimi"]

        if name not in students:
            continue

        if not count.get(name):
            count[name] = {}

        component = row["Komponent"]

        if not count[name].get(component):
            count[name][component] = 1
        else:
            count[name][component] += 1

    return count


def get_component_data(df: DataFrame, end_time: int, students: Set[str],
                       components: List[str], weeks: int,
                       label: int, web: int) -> Dict[str, List[Union[float, int]]]:
    data = {}
    count = get_component_count(df, end_time, students)

    for student in count:
        data[student] = []

        for component in components:
            value = count[student].get(component, 0) / weeks
            value = round(value, 2)
            data[student].append(value)

        data[student].append(label)
        data[student].append(web)

    return data


def get_score(df: DataFrame, students: List[str]) -> List[float]:
    scores = []
    scores_dict = {}

    for _, row in df.iterrows():
        scores_dict[f"{row['Eesnimi']} {row['Perenimi']}"] = row["Kogutulemus (Punktid)"]

    for student in students:
        score = scores_dict.get(student)
        scores.append(score)

    return scores


def get_grade(grade) -> float:
    return 0 if grade == "-" else grade


def get_activities_grade(df: DataFrame, students: List[str]) -> List[float]:
    grades = []
    grades_dict = {}

    for _, row in df.iterrows():
        grades_dict[f"{row['Eesnimi']} {row['Perenimi']}"] = 0

        for i in list(range(1,7)):
            grade = row[f"Test:{i}. nädala test (Punktid)"]
            grade = get_grade(grade)
            grades_dict[f"{row['Eesnimi']} {row['Perenimi']}"] += grade

        grades_dict[f"{row['Eesnimi']} {row['Perenimi']}"] = round(
            grades_dict[f"{row['Eesnimi']} {row['Perenimi']}"],
            2
        )

    for student in students:
        grade = grades_dict.get(student)
        grades.append(grade)

    return grades


def get_test_grade(df: DataFrame, students: List[str]) -> List[float]:
    grades = []
    grades_dict = {}

    for _, row in df.iterrows():
        grade1 = row["VPL harjutus:1. kontrolltöö programmide esitamine (Punktid)"]
        grade1 = get_grade(grade1)

        grade2 = row["VPL harjutus:1. KT järeltöö programmide esitamine (Punktid)"]
        grade2 = get_grade(grade2)

        grade3 = row["VPL harjutus:1. KT 5.01 lisajäreltöö programmide esitamine (Punktid)"]
        grade3 = get_grade(grade3)

        grade4 = row["VPL harjutus:1. KT 25.01 lisajäreltöö programmide esitamine (Punktid)"]
        grade4 = get_grade(grade4)

        grade = grade4 or grade3 or grade2 or grade1
        grades_dict[f"{row['Eesnimi']} {row['Perenimi']}"] = grade

    for student in students:
        grade = grades_dict.get(student)
        grades.append(grade)

    return grades


def get_labels() -> Dict[str, int]:
    all_labels = set()

    for path in os.listdir("../../xlsx/"):
        if not path.startswith("logs_") or not path.endswith(".xlsx"):
            continue

        speciality = get_speciality(path)
        all_labels.add(speciality)

    all_labels = list(all_labels)
    all_labels_dict = {}

    for i in range(len(all_labels)):
        all_labels_dict[all_labels[i]] = i + 1

    return all_labels_dict


def get_speciality(path: str) -> str:
    speciality = path.split('-')[1].split('_')[1]
    return speciality

In [33]:
import os


def save_csv(df_score: DataFrame, end_time: int, components: List[str], weeks: int):
    total_data_list = []
    total_activities = []
    total_test = []
    total_scores = []
    all_labels_dict = get_labels()

    for path in os.listdir("../../xlsx/"):
        if not path.startswith("logs_") or not path.endswith(".xlsx"):
            continue

        print(f"Processing {path}...")

        df = pd.read_excel(f"../../xlsx/{path}", engine="openpyxl", dtype=object, header=0)
        students = get_students(df, df_score)
        speciality = get_speciality(path)
        label = all_labels_dict[speciality]

        if "veebirühm" in path:
            web = 1
        else:
            web = 0

        data = get_component_data(df, end_time, students, components, weeks, label, web)
        names = list(data.keys())
        data_list = list(data.values())

        total_data_list = total_data_list + data_list

        activities = get_activities_grade(df_score, names)
        test = get_test_grade(df_score, names)
        scores = get_score(df_score, names)

        total_activities = total_activities + activities
        total_test = total_test + test
        total_scores = total_scores + scores

    total_data = DataFrame(total_data_list)
    total_data.columns = components + ["label"] + ["web"]

    total_data["activities"] = total_activities
    total_data["test"] = total_test
    total_data["y"] = total_scores

    total_data.to_csv("data.csv", encoding="utf-8", index=False)

    print("Done")


save_csv(
    df_score=dataframe_score,
    end_time=get_end_time(
        dataframe,
        get_students(
            dataframe,
            dataframe_score
        ),
        6
    ),
    components=get_components(dataframe),
    weeks=6
)

Processing logs_LTAT.03.001_20210207-0937_AT_02_veebirühm.xlsx...
Processing logs_LTAT.03.001_20210207-0947_Üld_01.xlsx...
Processing logs_LTAT.03.001_20210207-0942_Inf_04_veebirühm.xlsx...
Processing logs_LTAT.03.001_20210207-0943_Inf_05.xlsx...
Processing logs_LTAT.03.001_20210207-0943_Inf_06.xlsx...
Processing logs_LTAT.03.001_20210207-0946_Inf_10.xlsx...
Processing logs_LTAT.03.001_20210207-0942_Inf_03.xlsx...
Processing logs_LTAT.03.001_20210207-0938_FKM_01.xlsx...
Processing logs_LTAT.03.001_20210207-0938_FKM_02.xlsx...
Processing logs_LTAT.03.001_20210207-0940_FKM_03.xlsx...
Processing logs_LTAT.03.001_20210207-0946_Mat-stat_01.xlsx...
Processing logs_LTAT.03.001_20210207-0948_Üld_02.xlsx...
Processing logs_LTAT.03.001_20210207-0944_Inf_07.xlsx...
Processing logs_LTAT.03.001_20210207-0942_Inf_02.xlsx...
Processing logs_LTAT.03.001_20210207-0940_Inf_01.xlsx...
Processing logs_LTAT.03.001_20210207-0947_Mat-stat_02.xlsx...
Processing logs_LTAT.03.001_20210207-0937_AT_03.xlsx...
Pro